In [ ]:
import requests
import json
import prettytable
import openai

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0', 'SUUR0000SA0'], "startyear": "2011", "endyear": "2014"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

# Prompt the user for a query
query = input("Enter your Question: ")

# Prepare the data for the OpenAI GPT-3 model
documents = []
for series in json_data['Results']['series']:
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes = ""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            document = f"{seriesId} {year} {period} {value} {footnotes}"
            documents.append(document)

# Join the documents with new lines
documents_text = "\n".join(documents)

# Set up the OpenAI API credentials
openai.api_key = 'Your API key'

# Use the OpenAI GPT-3 model to answer the query
response = openai.Completion.create(
    engine='text-davinci-003',
    prompt=query,
    max_tokens=50,
    temperature=0.7,
    n=1,
    stop=None
)

# Print the answer
print(response.choices[0].text)
